In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os

In [2]:
tf.__version__

'2.3.0'

In [3]:
model = tf.keras.models.load_model("facenet_keras.h5")

In [4]:
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

In [8]:
def getImagemComId():
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    faces =[]
    ids = []
    for caminhoImagem in caminhos:
        imageFace = cv2.cvtColor(cv2.imread(caminhoImagem),cv2.COLOR_BGR2RGB)
        imageFace = cv2.resize(imageFace, (160, 160))
        id = int(os.path.split(caminhoImagem)[-1].split('.')[1])
        ids.append(id)
        emd = get_embedding(model, imageFace)
        faces.append(emd)
    return np.array(ids), faces

In [6]:
ids,faces = getImagemComId()

print('Embeddings...')

Embeddings...


In [10]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC


# normalize input vectors
in_encoder = Normalizer()
emdTrainX_norm = in_encoder.transform(faces)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(ids)
trainy_enc = out_encoder.transform(ids)
# fit model
modelSVC = SVC(kernel='linear', probability=True)
modelSVC.fit(emdTrainX_norm, trainy_enc)


# predict
yhat_train = modelSVC.predict(emdTrainX_norm)
# score
score_train = accuracy_score(trainy_enc, yhat_train)
# summarize
print(score_train)

1.0


In [24]:
faces = []
imageFace = cv2.cvtColor(cv2.imread("fotos//pessoa.3.1.jpg"),cv2.COLOR_BGR2RGB)
imageFace = cv2.resize(imageFace, (160, 160))
face = get_embedding(model, imageFace)
faces.append(face)
print(np.shape(faces))
in_encoder = Normalizer()
emdTrainX_norm = in_encoder.transform(faces)

yhat = modelSVC.predict(emdTrainX_norm)

yhat[0]



(1, 128)


2

In [11]:
import pickle
filename = 'SVC_model.sav'
pickle.dump(modelSVC, open(filename, 'wb'))
